In [1]:
from web3 import Web3
from web3.eth import Eth
from eth_account import Account
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.PublicKey import ECC
from Crypto.Hash import SHA256, SHA512
import struct
import base64
import hmac
import hashlib


In [2]:
BS = 16
def pad(data):
    padding = BS - len(data) % BS
    return data + padding * struct.pack("B", padding)

def unpad(data):
    return data[0:-data[-1]]

def AES256CbcDecrypt(hex_data, key=b'0'*32, iv=b'0'*16):
    data = bytearray.fromhex(hex_data)
    aes = AES.new(key, AES.MODE_CBC, iv)
    return unpad(aes.decrypt(data))

def AES256CbcEncrypt(bin_data, key=b'0'*32, iv=b'0'*16):
    aes = AES.new(key, AES.MODE_CBC, iv)
    return aes.encrypt(pad(bin_data))

In [3]:
#Test pading /unpading
start = bytearray(b'test')
padded = pad(start)
print(len(padded))
unpaded = unpad(padded)
print (unpaded)
print (unpaded==start)

16
bytearray(b'test')
True


In [4]:
#Test encrypt
hex_key = '1ac27dc3580fa322d795d54fe4b4b211980a299f924f1df267195c4f8253b700'
key = bytearray.fromhex(hex_key)
text ='test'
plaintext = text.encode(encoding='utf_8') 
hex_iv='95eb6bfc4ddcd00ba1b2d63b7c26bff4'
iv= bytearray.fromhex(hex_iv)

cry = AES256CbcEncrypt(plaintext,key,iv).hex()
print(cry)
print(cry=='e4bec3c272130e40fa34f068819d8e83')

#test decrypt
dec = AES256CbcDecrypt(cry,key,iv)
print(dec.decode("utf-8"))
print(dec.decode("utf-8")==text)

e4bec3c272130e40fa34f068819d8e83
True
test
True


# Version 1

In [5]:
def encode_key(pub_key):
    return pub_key.pointQ.x.to_bytes(32)+ pub_key.pointQ.y.to_bytes(32)

In [6]:
test_encoded = (74112620180903413835928205978618879887273296202846370176989995621597768248548).to_bytes(32,byteorder='big')+(70352546531620714376440768798610125065500941604960487601320179994127409968403).to_bytes(32,byteorder='big')
[ test_encoded[i] for i in range(len(test_encoded)) ] == [163, 218, 65, 170, 47, 136, 127, 159, 98, 77, 71, 242, 112, 244, 229, 36, 177, 2, 147, 111, 8, 233, 154, 255, 238, 244, 221, 88, 15, 2, 236, 228, 155, 138, 33, 87, 29, 185, 26, 29, 154, 176, 239, 200, 165, 137, 137, 181, 96, 137, 184, 159, 197, 133, 182, 251, 181, 179, 184, 113, 179, 141, 117, 19]

True

In [7]:
def decode_key(encoded_pub_key):
    x = encoded_pub_key[:32]
    y = encoded_pub_key[32:]
    return ECC.EccPoint(int.from_bytes(x,byteorder='big'), int.from_bytes(y,byteorder='big'))

In [8]:
ephemPrivKey = ECC.generate(curve='secp256r1')
encoded = encode_key(ephemPrivKey.public_key())
decode_key(encoded)

In [9]:
def derive(private_key, pub_ecc_point):
    return (pub_ecc_point*private_key.d).x

---------------------------------

In [10]:
def EncryptMessage_1(publicKey_hex, plainText_string):
    if publicKey_hex[:2]=='0x':
        publicKey_hex=publicKey_hex[2:]
    
    # Generate the temporary key 
    ephemPrivKey = ECC.generate(curve='secp256r1')
    ephemPubKey = ephemPrivKey.public_key()
    ephemPubKeyEncoded = encode_key(ephemPubKey)
    
    # Load the public key
    publicKey = bytearray.fromhex(publicKey_hex)
    pub = ECC.EccPoint(int.from_bytes(publicKey[:int(len(publicKey)/2)],byteorder='big'), int.from_bytes(publicKey[int(len(publicKey)/2):],byteorder='big'))
    
    # ECDH => get the shared secret
    px = derive(ephemPrivKey,pub) 
    
    # compute the encription and MAC keys
    hash_px = SHA512.new(data=px.to_bytes()).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    
    # cipher the plain text
    iv = Random.get_random_bytes(16)
    plaintext = plainText_string.encode(encoding='utf_8') 
    ciphertext = AES256CbcEncrypt(plaintext,encryptionKey,iv)
    
    # compute the MAC
    dataToMac = iv + bytearray([4])+ephemPubKeyEncoded + ciphertext
    mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    
    #build the output
    serializedCiphertext = iv + bytearray([4])+ephemPubKeyEncoded + mac + ciphertext
    return serializedCiphertext.hex()
    

In [11]:
def DecryptMessage_1(privateKey_hey, encrypted_hex):
    if privateKey_hey[:2]=='0x':
        privateKey_hey=privateKey_hey[2:]
    if encrypted_hex[:2]=='0x':
        encrypted_hex=encrypted_hex[2:]

    # get the components 
    encrypted = bytearray.fromhex(encrypted_hex)
    iv = encrypted[:16]
    ephemPubKeyEncoded = encrypted[17:81]
    mac = encrypted[81:113]
    ciphertext = encrypted[113:]
    
    # recover the temporary public key
    ephemPubKey = decode_key(ephemPubKeyEncoded)
    
    # load the private key
    privKey = ECC.construct(curve='secp256r1', d=int(privateKey_hey,16))
    
    # ECDH => get the shared secret
    px = derive(privKey, ephemPubKey)
    
    # compute the encription and MAC keys
    hash_px = SHA512.new(data=px.to_bytes()).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    
    # check the MAC
    dataToMac = iv + bytearray([4])+ephemPubKeyEncoded + ciphertext
    computed_mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    if computed_mac!=mac:
        raise ValueError("MAC missmatch")
        
    #decipher the text
    plaintext = AES256CbcDecrypt(ciphertext.hex(), encryptionKey, iv)
    return plaintext.decode("utf-8")
    

In [12]:
test_key_priv = ECC.generate(curve='secp256r1')
test_key_pub = test_key_priv.public_key()

priv_key_test_1 = hex(int(test_key_priv.d))
pub_key_test_1 = hex(int(test_key_pub.pointQ.x)) + hex(int(test_key_pub.pointQ.y))[2:]

print(priv_key_test_1)
print(pub_key_test_1)

0x5788560c124ec1956e650ab1b4c94ea5a456274735bb85c3f1b9daeafe57f51a
0x960a5328e1a81e1f2bdc7777f1e5551bbde80551cbc9d7bdce46855ae3f27cd46fb0e6117a38081efeaa69f9b40c24e370f8a2828f21e9d2968c1a0cb983aa82


In [13]:
message = 'test'
encrypted = EncryptMessage_1(pub_key_test_1, message)


mm = DecryptMessage_1(priv_key_test_1, encrypted)
print(mm)

test


# Version 2

In [14]:
from ecdsa import SigningKey, SECP256k1, ECDH, VerifyingKey



In [15]:
def EncryptMessage_2(publicKey_hex, plainText_string):
    if publicKey_hex[:2]=='0x':
        publicKey_hex=publicKey_hex[2:]
    publicKey_bin=bytearray.fromhex(publicKey_hex)
    
    # Generate the temporary key 
    ecdh = ECDH(curve=SECP256k1)
    ecdh.generate_private_key()
    
    ephemPubKeyEncoded = bytearray.fromhex(ecdh.get_public_key().to_string().hex()) 
    
    # Load the public key
    publicKey = VerifyingKey.from_string(publicKey_bin, curve=SECP256k1)
     
    # ECDH => get the shared secret
    ecdh.load_received_public_key(publicKey)
        
    px = ecdh.generate_sharedsecret_bytes()

    # compute the encription and MAC keys
    hash_px = SHA512.new(data=px).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    
    # cipher the plain text
    iv = Random.get_random_bytes(16)
    plaintext = plainText_string.encode(encoding='utf_8') 
    ciphertext = AES256CbcEncrypt(plaintext,encryptionKey,iv)
    
    # compute the MAC
    dataToMac = iv + bytearray([4])+ephemPubKeyEncoded + ciphertext
    mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    
    #build the output
    serializedCiphertext = iv + bytearray([4])+ephemPubKeyEncoded + mac + ciphertext
    return serializedCiphertext.hex()
    

In [16]:
def DecryptMessage_2(privateKey_hex, encrypted_hex):
    if privateKey_hex[:2]=='0x':
        privateKey_hex=privateKey_hex[2:]
    if encrypted_hex[:2]=='0x':
        encrypted_hex=encrypted_hex[2:]

    # get the components 
    encrypted = bytearray.fromhex(encrypted_hex)
    iv = encrypted[:16]
    ephemPubKeyEncoded = encrypted[17:81]
    mac = encrypted[81:113]
    ciphertext = encrypted[113:]
    
    # recover the temporary public key
    ephemPubKey = VerifyingKey.from_string(ephemPubKeyEncoded, curve=SECP256k1)
    
    # load the private key
    priv_key = SigningKey.from_secret_exponent(int(privateKey_hex,16), curve=SECP256k1)
    ecdh = ECDH(curve=SECP256k1, private_key=priv_key) 
    
    # ECDH => get the shared secret
    ecdh.load_received_public_key(ephemPubKey)
    px = ecdh.generate_sharedsecret_bytes()

    
    # compute the encription and MAC keys
    hash_px = SHA512.new(data=px).digest()
    encryptionKey = hash_px[:32]
    macKey = hash_px[32:]
    
    # check the MAC
    dataToMac = iv + bytearray([4])+ephemPubKeyEncoded + ciphertext
    computed_mac = hmac.new(macKey, dataToMac, 'sha256').digest()
    if computed_mac!=mac:
        raise ValueError("MAC missmatch")
        
    #decipher the text
    plaintext = AES256CbcDecrypt(ciphertext.hex(), encryptionKey, iv)
    return plaintext.decode("utf-8")
    

In [18]:
priv_key_test_2 =bytearray( [146, 123, 146, 246, 167, 182, 212, 139, 16, 46, 196, 210, 226, 21, 149, 250, 106, 205, 44, 197, 98, 4, 185, 16, 167, 174, 41, 219, 31, 104, 56, 212]).hex()
print('PRIV 0x'+priv_key_test_2)
key = SigningKey.from_secret_exponent(int(priv_key_test_2,16), curve=SECP256k1)
pub_key_test_2 = key.get_verifying_key().to_string().hex()
print('PUB 0x'+pub_key_test_2)


PRIV 0x927b92f6a7b6d48b102ec4d2e21595fa6acd2cc56204b910a7ae29db1f6838d4
PUB 0x0ae028e1095404fa6ff6dc35994abc99a73fa665792dd82464ad194f83e3ec60bd5b8d8972cfb29adad3cdd9276a8e78543f7f8a96aedc9ec82621fb8da52216


In [19]:
enc = EncryptMessage_2(pub_key_test_2,'Message')
print(DecryptMessage_2(priv_key_test_2, enc))

Message


# Cross compatibility test

In [20]:
message = 'test'
encrypted = EncryptMessage_1(pub_key_test_1, message)
print(DecryptMessage_2(priv_key_test_1, encrypted))

MalformedPointError: Point does not lie on the curve

In [21]:
encrypted = EncryptMessage_1(pub_key_test_2, message)
print(DecryptMessage_2(priv_key_test_2, encrypted))

MalformedPointError: Point does not lie on the curve

In [22]:
message = 'test'
EncryptMessage_2(pub_key_test_2, message)
print(DecryptMessage_1(priv_key_test_2, encrypted))

ValueError: MAC missmatch

In [23]:
message = 'test'
EncryptMessage_2(pub_key_test_1, message)
print(DecryptMessage_1(priv_key_test_1, encrypted))

MalformedPointError: Point does not lie on the curve

# Crosscompatibility with JS

In [24]:
private_JS_key  = bytearray( [116, 88, 207, 180, 148, 187, 167, 192, 59, 179, 153, 112, 164, 79, 236, 124, 90, 133, 130, 104, 97, 225, 49, 171, 185, 152, 115, 253, 143, 230, 77, 175]).hex()
pub_JS_key = bytearray([49, 176, 0, 75, 20, 225, 173, 169, 33, 197, 161, 167, 157, 12, 19, 37, 23, 114, 136, 233, 30, 191, 97, 191, 185, 24, 56, 152, 204, 69, 251, 52, 40, 109, 251, 14, 100, 245, 14, 56, 218, 217, 19, 70, 54, 192, 92, 19, 123, 11, 113, 69, 191, 171, 184, 218, 27, 138, 184, 36, 19, 28, 7, 164]).hex()
message_js_1 = "910df89676fecef464796ae011e224410449db12c818e33511c4341d16768e5ac62a6c4cc9913ecfc7e3de0f7bf7d1a68b95957c69c33d4b118dd21b105097ed7bb7f124915c510fa842f7c1570af6cfe548ba80ec2bd31cb26cf471a04d555a6c0ead4e5f367425ec2f22de5dc429fc1a653c54c24281165fb5db4fab6214af7f" 
message_js_2 = "b865cbd5f22ece7f568396dddcd9d45604c54c83c7447069c52bf59b686d0c4e6b511c22b24263b91ffd7d23da47eb3bd027d0684da8a06c1fd3a37aaf64fe9f6c180d1a5056cb11f1c7fae252d8232c182dbf3f89012af62c8afa4e83de656a996cdd996cf930b25e80ec4ff6f843694e8ac9fec521da9c8f0ab16765e04d136e"

In [25]:
enc = EncryptMessage_1(pub_JS_key,'Message')
print(DecryptMessage_1(private_JS_key, enc))

ValueError: MAC missmatch

In [26]:
enc = EncryptMessage_2(pub_JS_key,'Message')
print(DecryptMessage_2(private_JS_key, enc))

Message


In [27]:
DecryptMessage_1(private_JS_key, message_js_2)

ValueError: MAC missmatch

In [29]:
print(DecryptMessage_2(private_JS_key, message_js_1))
print(DecryptMessage_2(private_JS_key, message_js_2))

test split
Message
